In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score, plot_roc_curve

In [3]:
!ls


'NFL Gambling Modeling.ipynb'		 spreadspoke.R
'NFL Gambling Pre-Modeling Work.ipynb'	 spreadspoke_scores.csv
 nfl_stadiums.csv			 Untitled.ipynb
 nfl_teams.csv


In [4]:
df1 = pd.read_csv('spreadspoke_scores.csv')

In [5]:
df1 = df1.loc[(df1['schedule_season'] >= 1979) & (df1['schedule_week'] != 'Superbowl') 
              & (df1['stadium_neutral'] == False) & (df1['team_favorite_id'] != 'PICK')]

In [6]:
df1 = df1.drop(columns= 'stadium_neutral')

In [7]:
df1.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
2501,9/1/1979,1979,1,False,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,TB,-3.0,30,Houlihan's Stadium,79.0,9.0,87.0,NaN
2502,9/2/1979,1979,1,False,Buffalo Bills,7.0,9.0,Miami Dolphins,MIA,-5.0,39,Ralph Wilson Stadium,74.0,15.0,74.0,NaN
2503,9/2/1979,1979,1,False,Chicago Bears,6.0,3.0,Green Bay Packers,CHI,-3.0,31,Soldier Field,78.0,11.0,68.0,NaN
2504,9/2/1979,1979,1,False,Denver Broncos,10.0,0.0,Cincinnati Bengals,DEN,-3.0,31.5,Mile High Stadium,69.0,6.0,38.0,NaN
2505,9/2/1979,1979,1,False,Kansas City Chiefs,14.0,0.0,Baltimore Colts,KC,-1.0,37,Arrowhead Stadium,76.0,8.0,71.0,NaN


In [8]:
len(df1)

10498

In [9]:
df1.describe()

,schedule_season,score_home,score_away,spread_favorite,weather_temperature,weather_wind_mph,weather_humidity
count,10498.000000,10498.000000,10498.000000,10498.000000,9694.000000,9694.000000,6223.000000
mean,2001.010859,22.791960,20.050391,-5.465089,59.912833,7.206519,66.515186
std,12.255204,10.377662,10.081200,3.399308,15.475054,5.741684,16.095935
min,1979.000000,0.000000,0.000000,-26.500000,-6.000000,0.000000,4.000000
25%,1991.000000,16.000000,13.000000,-7.000000,49.000000,0.000000,56.000000
50%,2002.000000,23.000000,20.000000,-4.500000,64.000000,7.000000,68.000000
75%,2012.000000,30.000000,27.000000,-3.000000,72.000000,11.000000,78.000000
max,2021.000000,62.000000,59.000000,-1.000000,97.000000,40.000000,100.000000


In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10498 entries, 2501 to 13230
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   schedule_date        10498 non-null  object 
 1   schedule_season      10498 non-null  int64  
 2   schedule_week        10498 non-null  object 
 3   schedule_playoff     10498 non-null  bool   
 4   team_home            10498 non-null  object 
 5   score_home           10498 non-null  float64
 6   score_away           10498 non-null  float64
 7   team_away            10498 non-null  object 
 8   team_favorite_id     10498 non-null  object 
 9   spread_favorite      10498 non-null  float64
 10  over_under_line      10498 non-null  object 
 11  stadium              10498 non-null  object 
 12  weather_temperature  9694 non-null   float64
 13  weather_wind_mph     9694 non-null   float64
 14  weather_humidity     6223 non-null   float64
 15  weather_detail       2586 non-nul

In [11]:
home_teams = df1['team_home']

In [12]:
home_team_ab = []
for team in home_teams:
    if team.__contains__('Cardinals'):
        home_team_ab.append('ARI')
    elif team.__contains__('Falcons'):
        home_team_ab.append('ATL')
    elif team.__contains__('Ravens'):
        home_team_ab.append('BAL')
    elif team.__contains__('Bills'):
        home_team_ab.append('BUF')
    elif team.__contains__('Panthers'):
        home_team_ab.append('CAR')
    elif team.__contains__('Bears'):
        home_team_ab.append('CHI')
    elif team.__contains__('Bengals'):
        home_team_ab.append('CIN')
    elif team.__contains__('Browns'):
        home_team_ab.append('CLE')
    elif team.__contains__('Cowboys'):
        home_team_ab.append('DAL')
    elif team.__contains__('Broncos'):
        home_team_ab.append('DEN')
    elif team.__contains__('Lions'):
        home_team_ab.append('DET')
    elif team.__contains__('Packers'):
        home_team_ab.append('GB')
    elif team.__contains__('Texans'):
        home_team_ab.append('HOU')
    elif team.__contains__('Colts'):
        home_team_ab.append('IND')
    elif team.__contains__('Jaguars'):
        home_team_ab.append('JAX')
    elif team.__contains__('Chiefs'):
        home_team_ab.append('KC')
    elif team.__contains__('Chargers'):
        home_team_ab.append('LAC')
    elif team.__contains__('Rams'):
        home_team_ab.append('LAR')
    elif team.__contains__('Dolphins'):
        home_team_ab.append('MIA')
    elif team.__contains__('Vikings'):
        home_team_ab.append('MIN')
    elif team.__contains__('Patriots'):
        home_team_ab.append('NE')
    elif team.__contains__('Saints'):
        home_team_ab.append('NO')
    elif team.__contains__('Giants'):
        home_team_ab.append('NYG')
    elif team.__contains__('Jets'):
        home_team_ab.append('NYJ')
    elif team.__contains__('Raiders'):
        home_team_ab.append('LVR')
    elif team.__contains__('Eagles'):
        home_team_ab.append('PHI')
    elif team.__contains__('Steelers'):
        home_team_ab.append('PIT')
    elif team.__contains__('Seahawks'):
        home_team_ab.append('SEA')
    elif team.__contains__('49ers'):
        home_team_ab.append('SF')
    elif team.__contains__('Buccaneers'):
        home_team_ab.append('TB')
    elif team.__contains__('Oilers'):
        home_team_ab.append('TEN')
    elif team.__contains__('Titans'):
        home_team_ab.append('TEN')
    elif team.__contains__('Redskins'):
        home_team_ab.append('WAS')
    elif team.__contains__('Football Team'):
        home_team_ab.append('WAS')

In [13]:
df1['team_home_id'] = home_team_ab
df1

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_id
2501,9/1/1979,1979,1,False,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,TB,-3.0,30,Houlihan's Stadium,79.0,9.0,87.0,NaN,TB
2502,9/2/1979,1979,1,False,Buffalo Bills,7.0,9.0,Miami Dolphins,MIA,-5.0,39,Ralph Wilson Stadium,74.0,15.0,74.0,NaN,BUF
2503,9/2/1979,1979,1,False,Chicago Bears,6.0,3.0,Green Bay Packers,CHI,-3.0,31,Soldier Field,78.0,11.0,68.0,NaN,CHI
2504,9/2/1979,1979,1,False,Denver Broncos,10.0,0.0,Cincinnati Bengals,DEN,-3.0,31.5,Mile High Stadium,69.0,6.0,38.0,NaN,DEN
2505,9/2/1979,1979,1,False,Kansas City Chiefs,14.0,0.0,Baltimore Colts,KC,-1.0,37,Arrowhead Stadium,76.0,8.0,71.0,NaN,KC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,1/22/2022,2021,Division,True,Tennessee Titans,16.0,19.0,Cincinnati Bengals,TEN,-4.0,48.5,Nissan Stadium,35.0,0.0,43.0,NaN,TEN
13227,1/23/2022,2021,Division,True,Kansas City Chiefs,42.0,36.0,Buffalo Bills,KC,-2.5,54.0,Arrowhead Stadium,35.0,6.0,54.0,NaN,KC
13228,1/23/2022,2021,Division,True,Tampa Bay Buccaneers,27.0,30.0,Los Angeles Rams,TB,-3.0,48.0,Raymond James Stadium,50.0,11.0,69.0,NaN,TB
13229,1/30/2022,2021,Conference,True,Kansas City Chiefs,24.0,27.0,Cincinnati Bengals,KC,-7.0,54.5,Arrowhead Stadium,41.0,4.0,41.0,NaN,KC


In [14]:
away_teams = list(df1['team_away'].values)

In [15]:
away_team_ab = []
for team in away_teams:
    if team.__contains__('Cardinals'):
        away_team_ab.append('ARI')
    elif team.__contains__('Falcons'):
        away_team_ab.append('ATL')
    elif team.__contains__('Ravens'):
        away_team_ab.append('BAL')
    elif team.__contains__('Bills'):
        away_team_ab.append('BUF')
    elif team.__contains__('Panthers'):
        away_team_ab.append('CAR')
    elif team.__contains__('Bears'):
        away_team_ab.append('CHI')
    elif team.__contains__('Bengals'):
        away_team_ab.append('CIN')
    elif team.__contains__('Browns'):
        away_team_ab.append('CLE')
    elif team.__contains__('Cowboys'):
        away_team_ab.append('DAL')
    elif team.__contains__('Broncos'):
        away_team_ab.append('DEN')
    elif team.__contains__('Lions'):
        away_team_ab.append('DET')
    elif team.__contains__('Packers'):
        away_team_ab.append('GB')
    elif team.__contains__('Texans'):
        away_team_ab.append('HOU')
    elif team.__contains__('Colts'):
        away_team_ab.append('IND')
    elif team.__contains__('Jaguars'):
        away_team_ab.append('JAX')
    elif team.__contains__('Chiefs'):
        away_team_ab.append('KC')
    elif team.__contains__('Chargers'):
        away_team_ab.append('LAC')
    elif team.__contains__('Rams'):
        away_team_ab.append('LAR')
    elif team.__contains__('Dolphins'):
        away_team_ab.append('MIA')
    elif team.__contains__('Vikings'):
        away_team_ab.append('MIN')
    elif team.__contains__('Patriots'):
        away_team_ab.append('NE')
    elif team.__contains__('Saints'):
        away_team_ab.append('NO')
    elif team.__contains__('Giants'):
        away_team_ab.append('NYG')
    elif team.__contains__('Jets'):
        away_team_ab.append('NYJ')
    elif team.__contains__('Raiders'):
        away_team_ab.append('LVR')
    elif team.__contains__('Eagles'):
        away_team_ab.append('PHI')
    elif team.__contains__('Steelers'):
        away_team_ab.append('PIT')
    elif team.__contains__('Seahawks'):
        away_team_ab.append('SEA')
    elif team.__contains__('49ers'):
        away_team_ab.append('SF')
    elif team.__contains__('Buccaneers'):
        away_team_ab.append('TB')
    elif team.__contains__('Oilers'):
        away_team_ab.append('TEN')
    elif team.__contains__('Titans'):
        away_team_ab.append('TEN')
    elif team.__contains__('Redskins'):
        away_team_ab.append('WAS')
    elif team.__contains__('Football Team'):
        away_team_ab.append('WAS')

In [16]:
df1['team_away_id'] = away_team_ab

In [17]:
df1['team_away_id']

2501     DET
2502     MIA
2503      GB
2504     CIN
2505     IND
        ... 
13226    CIN
13227    BUF
13228    LAR
13229    CIN
13230     SF
Name: team_away_id, Length: 10498, dtype: object

In [18]:
df1.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_id,team_away_id
2501,9/1/1979,1979,1,False,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,TB,-3.0,30,Houlihan's Stadium,79.0,9.0,87.0,NaN,TB,DET
2502,9/2/1979,1979,1,False,Buffalo Bills,7.0,9.0,Miami Dolphins,MIA,-5.0,39,Ralph Wilson Stadium,74.0,15.0,74.0,NaN,BUF,MIA
2503,9/2/1979,1979,1,False,Chicago Bears,6.0,3.0,Green Bay Packers,CHI,-3.0,31,Soldier Field,78.0,11.0,68.0,NaN,CHI,GB
2504,9/2/1979,1979,1,False,Denver Broncos,10.0,0.0,Cincinnati Bengals,DEN,-3.0,31.5,Mile High Stadium,69.0,6.0,38.0,NaN,DEN,CIN
2505,9/2/1979,1979,1,False,Kansas City Chiefs,14.0,0.0,Baltimore Colts,KC,-1.0,37,Arrowhead Stadium,76.0,8.0,71.0,NaN,KC,IND


In [19]:
score_home = list(df1['score_home'].values)
score_away = list(df1['score_away'].values)

In [20]:
n_home_teams = list(df1['team_home_id'].values)

In [21]:
n_away_teams = list(df1['team_away_id'].values)

In [22]:
fav_team = list(df1['team_favorite_id'].values)

In [23]:
favorite_win = []
for score in list(range(0,10498)):
    if score_home[score] > score_away[score]:
        if n_home_teams[score] == fav_team[score]:
            favorite_win.append(True)
        else:
            favorite_win.append(False)
    elif score_home[score] < score_away[score]:
        if n_away_teams[score] == fav_team[score]:
            favorite_win.append(True)
        else:
            favorite_win.append(False)
    elif score_home[score] == score_away[score]:
        favorite_win.append(False)

In [24]:
favorite_win

[True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,


In [25]:
df1['favorite_win'] = favorite_win

In [26]:
df1.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_id,team_away_id,favorite_win
2501,9/1/1979,1979,1,False,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,TB,-3.0,30,Houlihan's Stadium,79.0,9.0,87.0,NaN,TB,DET,True
2502,9/2/1979,1979,1,False,Buffalo Bills,7.0,9.0,Miami Dolphins,MIA,-5.0,39,Ralph Wilson Stadium,74.0,15.0,74.0,NaN,BUF,MIA,True
2503,9/2/1979,1979,1,False,Chicago Bears,6.0,3.0,Green Bay Packers,CHI,-3.0,31,Soldier Field,78.0,11.0,68.0,NaN,CHI,GB,True
2504,9/2/1979,1979,1,False,Denver Broncos,10.0,0.0,Cincinnati Bengals,DEN,-3.0,31.5,Mile High Stadium,69.0,6.0,38.0,NaN,DEN,CIN,True
2505,9/2/1979,1979,1,False,Kansas City Chiefs,14.0,0.0,Baltimore Colts,KC,-1.0,37,Arrowhead Stadium,76.0,8.0,71.0,NaN,KC,IND,True


In [27]:
df1['home_favorite'] = list((df1['team_home_id'] == df1['team_favorite_id']).values)

In [28]:
df1.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_id,team_away_id,favorite_win,home_favorite
2501,9/1/1979,1979,1,False,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,TB,-3.0,30,Houlihan's Stadium,79.0,9.0,87.0,NaN,TB,DET,True,True
2502,9/2/1979,1979,1,False,Buffalo Bills,7.0,9.0,Miami Dolphins,MIA,-5.0,39,Ralph Wilson Stadium,74.0,15.0,74.0,NaN,BUF,MIA,True,False
2503,9/2/1979,1979,1,False,Chicago Bears,6.0,3.0,Green Bay Packers,CHI,-3.0,31,Soldier Field,78.0,11.0,68.0,NaN,CHI,GB,True,True
2504,9/2/1979,1979,1,False,Denver Broncos,10.0,0.0,Cincinnati Bengals,DEN,-3.0,31.5,Mile High Stadium,69.0,6.0,38.0,NaN,DEN,CIN,True,True
2505,9/2/1979,1979,1,False,Kansas City Chiefs,14.0,0.0,Baltimore Colts,KC,-1.0,37,Arrowhead Stadium,76.0,8.0,71.0,NaN,KC,IND,True,True


In [29]:
df1['weather_temperature'].min()

-6.0

In [30]:
df1['schedule_date'] = pd.to_datetime(df1['schedule_date'])

In [31]:
df1['schedule_date'].value_counts()

2010-01-03    16
2014-12-28    16
2018-12-30    16
2017-01-01    16
2016-01-03    16
              ..
2004-12-24     1
1997-12-01     1
1989-12-04     1
1981-12-07     1
2014-12-11     1
Name: schedule_date, Length: 1973, dtype: int64

In [32]:
df1['yyyy'] = pd.to_datetime(df1['schedule_date']).dt.year
df1['mm'] = pd.to_datetime(df1['schedule_date']).dt.month

In [33]:
df1.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_id,team_away_id,favorite_win,home_favorite,yyyy,mm
2501,1979-09-01,1979,1,False,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,TB,-3.0,...,79.0,9.0,87.0,NaN,TB,DET,True,True,1979,9
2502,1979-09-02,1979,1,False,Buffalo Bills,7.0,9.0,Miami Dolphins,MIA,-5.0,...,74.0,15.0,74.0,NaN,BUF,MIA,True,False,1979,9
2503,1979-09-02,1979,1,False,Chicago Bears,6.0,3.0,Green Bay Packers,CHI,-3.0,...,78.0,11.0,68.0,NaN,CHI,GB,True,True,1979,9
2504,1979-09-02,1979,1,False,Denver Broncos,10.0,0.0,Cincinnati Bengals,DEN,-3.0,...,69.0,6.0,38.0,NaN,DEN,CIN,True,True,1979,9
2505,1979-09-02,1979,1,False,Kansas City Chiefs,14.0,0.0,Baltimore Colts,KC,-1.0,...,76.0,8.0,71.0,NaN,KC,IND,True,True,1979,9


In [34]:
df1['mm'].value_counts()

12    2632
11    2598
10    2418
9     2261
1      575
8       14
Name: mm, dtype: int64

In [35]:
df1.loc[df1['mm'] == 1]

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_id,team_away_id,favorite_win,home_favorite,yyyy,mm
2731,1980-01-06,1979,Conference,True,Pittsburgh Steelers,27.0,13.0,Houston Oilers,PIT,-9.5,...,NaN,NaN,NaN,NaN,PIT,TEN,True,True,1980,1
2732,1980-01-06,1979,Conference,True,Tampa Bay Buccaneers,0.0,9.0,Los Angeles Rams,LAR,-3.5,...,NaN,NaN,NaN,NaN,TB,LAR,True,False,1980,1
2960,1981-01-03,1980,Division,True,Philadelphia Eagles,31.0,16.0,Minnesota Vikings,PHI,-7.0,...,NaN,NaN,NaN,NaN,PHI,MIN,True,True,1981,1
2961,1981-01-03,1980,Division,True,San Diego Chargers,20.0,14.0,Buffalo Bills,LAC,-6.0,...,NaN,NaN,NaN,NaN,LAC,BUF,True,True,1981,1
2962,1981-01-04,1980,Division,True,Atlanta Falcons,27.0,30.0,Dallas Cowboys,ATL,-2.5,...,NaN,NaN,NaN,NaN,ATL,DAL,False,True,1981,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,2022-01-22,2021,Division,True,Tennessee Titans,16.0,19.0,Cincinnati Bengals,TEN,-4.0,...,35.0,0.0,43.0,NaN,TEN,CIN,False,True,2022,1
13227,2022-01-23,2021,Division,True,Kansas City Chiefs,42.0,36.0,Buffalo Bills,KC,-2.5,...,35.0,6.0,54.0,NaN,KC,BUF,True,True,2022,1
13228,2022-01-23,2021,Division,True,Tampa Bay Buccaneers,27.0,30.0,Los Angeles Rams,TB,-3.0,...,50.0,11.0,69.0,NaN,TB,LAR,False,True,2022,1
13229,2022-01-30,2021,Conference,True,Kansas City Chiefs,24.0,27.0,Cincinnati Bengals,KC,-7.0,...,41.0,4.0,41.0,NaN,KC,CIN,False,True,2022,1


In [36]:
weather_df = df1[['schedule_week', 'team_home', 'score_home', 'score_away', 'team_away', 'weather_temperature', 'weather_wind_mph', 'weather_humidity', 'favorite_win', 'home_favorite', 'yyyy', 'mm', 'team_home_id', 'team_away_id']]

In [37]:
weather_df.head()

,schedule_week,team_home,score_home,score_away,team_away,weather_temperature,weather_wind_mph,weather_humidity,favorite_win,home_favorite,yyyy,mm,team_home_id,team_away_id
2501,1,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,79.0,9.0,87.0,True,True,1979,9,TB,DET
2502,1,Buffalo Bills,7.0,9.0,Miami Dolphins,74.0,15.0,74.0,True,False,1979,9,BUF,MIA
2503,1,Chicago Bears,6.0,3.0,Green Bay Packers,78.0,11.0,68.0,True,True,1979,9,CHI,GB
2504,1,Denver Broncos,10.0,0.0,Cincinnati Bengals,69.0,6.0,38.0,True,True,1979,9,DEN,CIN
2505,1,Kansas City Chiefs,14.0,0.0,Baltimore Colts,76.0,8.0,71.0,True,True,1979,9,KC,IND


In [38]:
tb_weather_9 = weather_df.loc[
    (weather_df['team_home'] == 'TB') &
    (weather_df['mm'] == 9)
]
tb_weather_9['weather_temperature'].mean()

nan

In [39]:
type(df1['schedule_date'].iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [40]:
type(weather_df['mm'].iloc[0])

numpy.int64

In [41]:
weather_df

,schedule_week,team_home,score_home,score_away,team_away,weather_temperature,weather_wind_mph,weather_humidity,favorite_win,home_favorite,yyyy,mm,team_home_id,team_away_id
2501,1,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,79.0,9.0,87.0,True,True,1979,9,TB,DET
2502,1,Buffalo Bills,7.0,9.0,Miami Dolphins,74.0,15.0,74.0,True,False,1979,9,BUF,MIA
2503,1,Chicago Bears,6.0,3.0,Green Bay Packers,78.0,11.0,68.0,True,True,1979,9,CHI,GB
2504,1,Denver Broncos,10.0,0.0,Cincinnati Bengals,69.0,6.0,38.0,True,True,1979,9,DEN,CIN
2505,1,Kansas City Chiefs,14.0,0.0,Baltimore Colts,76.0,8.0,71.0,True,True,1979,9,KC,IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,Division,Tennessee Titans,16.0,19.0,Cincinnati Bengals,35.0,0.0,43.0,False,True,2022,1,TEN,CIN
13227,Division,Kansas City Chiefs,42.0,36.0,Buffalo Bills,35.0,6.0,54.0,True,True,2022,1,KC,BUF
13228,Division,Tampa Bay Buccaneers,27.0,30.0,Los Angeles Rams,50.0,11.0,69.0,False,True,2022,1,TB,LAR
13229,Conference,Kansas City Chiefs,24.0,27.0,Cincinnati Bengals,41.0,4.0,41.0,False,True,2022,1,KC,CIN


In [42]:
t_df = weather_df.groupby(['team_home', 'mm'])['weather_temperature'].mean()

In [43]:
t_df

team_home            mm
Arizona Cardinals    1     70.222222
                     9     77.673913
                     10    73.338983
                     11    67.490196
                     12    65.688525
                             ...    
Washington Redskins  1     47.142857
                     9     71.506667
                     10    59.944444
                     11    49.025000
                     12    41.766234
Name: weather_temperature, Length: 221, dtype: float64

In [44]:
t_df['Arizona Cardinals']

mm
1     70.222222
9     77.673913
10    73.338983
11    67.490196
12    65.688525
Name: weather_temperature, dtype: float64

In [45]:
def get_difference_in_temp(home_team, month, temp):
    diff = t_df[home_team][month] - temp
    return diff

In [46]:
differences = []
for team, m, t in weather_df[['team_home', 'mm', 'weather_temperature']].values:
    differences.append(get_difference_in_temp(team, m, t))

In [47]:
differences

[2.7285714285714278,
 -10.287500000000001,
 -11.683544303797461,
 -2.2249999999999943,
 -6.957746478873233,
 1.382352941176464,
 1.3670886075949369,
 0.0,
 -2.0434782608695627,
 -6.414285714285711,
 -4.470588235294116,
 -7.933333333333337,
 -8.49333333333334,
 -12.845070422535215,
 -0.22499999999999432,
 9.712499999999999,
 6.316455696202539,
 2.911392405063289,
 1.8548387096774235,
 6.1549295774647845,
 5.753246753246756,
 10.014705882352942,
 -1.78125,
 -0.5483870967741922,
 4.585714285714289,
 1.3380281690140805,
 9.696969696969703,
 7.303030303030297,
 9.470588235294116,
 2.911392405063289,
 0.0,
 -11.617647058823536,
 7.367088607594937,
 0.0,
 5.956521739130437,
 -7.78125,
 -4.470588235294116,
 5.066666666666663,
 3.5066666666666606,
 13.712499999999999,
 11.696969696969703,
 -2.2249999999999943,
 0.0,
 4.042253521126767,
 3.8548387096774235,
 12.367088607594937,
 10.154929577464785,
 7.753246753246756,
 11.014705882352942,
 7.066666666666663,
 -0.27142857142857224,
 10.3030303030

In [48]:
weather_df['away_temp_diff'] = differences
weather_df

<ipython-input-48-b140267ae5b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['away_temp_diff'] = differences


,schedule_week,team_home,score_home,score_away,team_away,weather_temperature,weather_wind_mph,weather_humidity,favorite_win,home_favorite,yyyy,mm,team_home_id,team_away_id,away_temp_diff
2501,1,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,79.0,9.0,87.0,True,True,1979,9,TB,DET,2.728571
2502,1,Buffalo Bills,7.0,9.0,Miami Dolphins,74.0,15.0,74.0,True,False,1979,9,BUF,MIA,-10.287500
2503,1,Chicago Bears,6.0,3.0,Green Bay Packers,78.0,11.0,68.0,True,True,1979,9,CHI,GB,-11.683544
2504,1,Denver Broncos,10.0,0.0,Cincinnati Bengals,69.0,6.0,38.0,True,True,1979,9,DEN,CIN,-2.225000
2505,1,Kansas City Chiefs,14.0,0.0,Baltimore Colts,76.0,8.0,71.0,True,True,1979,9,KC,IND,-6.957746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,Division,Tennessee Titans,16.0,19.0,Cincinnati Bengals,35.0,0.0,43.0,False,True,2022,1,TEN,CIN,9.800000
13227,Division,Kansas City Chiefs,42.0,36.0,Buffalo Bills,35.0,6.0,54.0,True,True,2022,1,KC,BUF,-1.000000
13228,Division,Tampa Bay Buccaneers,27.0,30.0,Los Angeles Rams,50.0,11.0,69.0,False,True,2022,1,TB,LAR,14.500000
13229,Conference,Kansas City Chiefs,24.0,27.0,Cincinnati Bengals,41.0,4.0,41.0,False,True,2022,1,KC,CIN,-7.000000


In [49]:
t_df['Miami Dolphins']

mm
1     73.500000
8     81.000000
9     82.854839
10    80.438356
11    75.040000
12    71.797872
Name: weather_temperature, dtype: float64

In [50]:
t_df['Buffalo Bills']

mm
1     28.142857
8     65.000000
9     63.712500
10    52.533333
11    43.050633
12    33.090909
Name: weather_temperature, dtype: float64

In [51]:
weather_df.loc[(weather_df['team_home_id'] == 'MIA') & (weather_df['mm'] == 9)]

,schedule_week,team_home,score_home,score_away,team_away,weather_temperature,weather_wind_mph,weather_humidity,favorite_win,home_favorite,yyyy,mm,team_home_id,team_away_id,away_temp_diff
2522,2,Miami Dolphins,19.0,10.0,Seattle Seahawks,81.0,13.0,81.0,True,True,1979,9,MIA,SEA,1.854839
2548,4,Miami Dolphins,31.0,16.0,Chicago Bears,79.0,8.0,86.0,True,True,1979,9,MIA,CHI,3.854839
2755,2,Miami Dolphins,17.0,16.0,Cincinnati Bengals,77.0,8.0,93.0,True,True,1980,9,MIA,CIN,5.854839
2782,4,Miami Dolphins,21.0,16.0,New Orleans Saints,85.0,10.0,72.0,True,True,1980,9,MIA,NO,-2.145161
2981,2,Miami Dolphins,30.0,10.0,Pittsburgh Steelers,81.0,10.0,80.0,True,True,1981,9,MIA,PIT,1.854839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12418,1,Miami Dolphins,10.0,59.0,Baltimore Ravens,NaN,NaN,NaN,True,False,2019,9,MIA,BAL,NaN
12436,2,Miami Dolphins,0.0,43.0,New England Patriots,NaN,NaN,NaN,True,False,2019,9,MIA,NE,NaN
12470,4,Miami Dolphins,10.0,30.0,Los Angeles Chargers,NaN,NaN,NaN,True,False,2019,9,MIA,LAC,NaN
12702,2,Miami Dolphins,28.0,31.0,Buffalo Bills,NaN,NaN,NaN,True,False,2020,9,MIA,BUF,NaN


In [52]:
weather_df

,schedule_week,team_home,score_home,score_away,team_away,weather_temperature,weather_wind_mph,weather_humidity,favorite_win,home_favorite,yyyy,mm,team_home_id,team_away_id,away_temp_diff
2501,1,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,79.0,9.0,87.0,True,True,1979,9,TB,DET,2.728571
2502,1,Buffalo Bills,7.0,9.0,Miami Dolphins,74.0,15.0,74.0,True,False,1979,9,BUF,MIA,-10.287500
2503,1,Chicago Bears,6.0,3.0,Green Bay Packers,78.0,11.0,68.0,True,True,1979,9,CHI,GB,-11.683544
2504,1,Denver Broncos,10.0,0.0,Cincinnati Bengals,69.0,6.0,38.0,True,True,1979,9,DEN,CIN,-2.225000
2505,1,Kansas City Chiefs,14.0,0.0,Baltimore Colts,76.0,8.0,71.0,True,True,1979,9,KC,IND,-6.957746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,Division,Tennessee Titans,16.0,19.0,Cincinnati Bengals,35.0,0.0,43.0,False,True,2022,1,TEN,CIN,9.800000
13227,Division,Kansas City Chiefs,42.0,36.0,Buffalo Bills,35.0,6.0,54.0,True,True,2022,1,KC,BUF,-1.000000
13228,Division,Tampa Bay Buccaneers,27.0,30.0,Los Angeles Rams,50.0,11.0,69.0,False,True,2022,1,TB,LAR,14.500000
13229,Conference,Kansas City Chiefs,24.0,27.0,Cincinnati Bengals,41.0,4.0,41.0,False,True,2022,1,KC,CIN,-7.000000


In [53]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10498 entries, 2501 to 13230
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   schedule_week        10498 non-null  object 
 1   team_home            10498 non-null  object 
 2   score_home           10498 non-null  float64
 3   score_away           10498 non-null  float64
 4   team_away            10498 non-null  object 
 5   weather_temperature  9694 non-null   float64
 6   weather_wind_mph     9694 non-null   float64
 7   weather_humidity     6223 non-null   float64
 8   favorite_win         10498 non-null  bool   
 9   home_favorite        10498 non-null  bool   
 10  yyyy                 10498 non-null  int64  
 11  mm                   10498 non-null  int64  
 12  team_home_id         10498 non-null  object 
 13  team_away_id         10498 non-null  object 
 14  away_temp_diff       9694 non-null   float64
dtypes: bool(2), float64(6), int64(2),

In [54]:
weather_df

,schedule_week,team_home,score_home,score_away,team_away,weather_temperature,weather_wind_mph,weather_humidity,favorite_win,home_favorite,yyyy,mm,team_home_id,team_away_id,away_temp_diff
2501,1,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,79.0,9.0,87.0,True,True,1979,9,TB,DET,2.728571
2502,1,Buffalo Bills,7.0,9.0,Miami Dolphins,74.0,15.0,74.0,True,False,1979,9,BUF,MIA,-10.287500
2503,1,Chicago Bears,6.0,3.0,Green Bay Packers,78.0,11.0,68.0,True,True,1979,9,CHI,GB,-11.683544
2504,1,Denver Broncos,10.0,0.0,Cincinnati Bengals,69.0,6.0,38.0,True,True,1979,9,DEN,CIN,-2.225000
2505,1,Kansas City Chiefs,14.0,0.0,Baltimore Colts,76.0,8.0,71.0,True,True,1979,9,KC,IND,-6.957746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,Division,Tennessee Titans,16.0,19.0,Cincinnati Bengals,35.0,0.0,43.0,False,True,2022,1,TEN,CIN,9.800000
13227,Division,Kansas City Chiefs,42.0,36.0,Buffalo Bills,35.0,6.0,54.0,True,True,2022,1,KC,BUF,-1.000000
13228,Division,Tampa Bay Buccaneers,27.0,30.0,Los Angeles Rams,50.0,11.0,69.0,False,True,2022,1,TB,LAR,14.500000
13229,Conference,Kansas City Chiefs,24.0,27.0,Cincinnati Bengals,41.0,4.0,41.0,False,True,2022,1,KC,CIN,-7.000000


In [55]:
wind_df = weather_df.groupby(['team_home', 'mm'])['weather_wind_mph'].mean()
wind_df

team_home            mm
Arizona Cardinals    1     0.666667
                     9     2.891304
                     10    3.423729
                     11    3.901961
                     12    2.426230
                             ...   
Washington Redskins  1     8.000000
                     9     8.280000
                     10    9.375000
                     11    9.137500
                     12    9.103896
Name: weather_wind_mph, Length: 221, dtype: float64

In [56]:
def get_difference_in_wind(home_team, month, wind):
    diff = wind_df[home_team][month] - wind
    return diff

In [57]:
differences = []
for team, w, wi in weather_df[['team_home', 'mm', 'weather_wind_mph']].values:
    differences.append(get_difference_in_wind(team, w, wi))

In [58]:
weather_df['away_wind_diff'] = differences
weather_df

<ipython-input-58-35761b0da99c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['away_wind_diff'] = differences


,schedule_week,team_home,score_home,score_away,team_away,weather_temperature,weather_wind_mph,weather_humidity,favorite_win,home_favorite,yyyy,mm,team_home_id,team_away_id,away_temp_diff,away_wind_diff
2501,1,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,79.0,9.0,87.0,True,True,1979,9,TB,DET,2.728571,-1.585714
2502,1,Buffalo Bills,7.0,9.0,Miami Dolphins,74.0,15.0,74.0,True,False,1979,9,BUF,MIA,-10.287500,-6.012500
2503,1,Chicago Bears,6.0,3.0,Green Bay Packers,78.0,11.0,68.0,True,True,1979,9,CHI,GB,-11.683544,-2.151899
2504,1,Denver Broncos,10.0,0.0,Cincinnati Bengals,69.0,6.0,38.0,True,True,1979,9,DEN,CIN,-2.225000,3.600000
2505,1,Kansas City Chiefs,14.0,0.0,Baltimore Colts,76.0,8.0,71.0,True,True,1979,9,KC,IND,-6.957746,1.295775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,Division,Tennessee Titans,16.0,19.0,Cincinnati Bengals,35.0,0.0,43.0,False,True,2022,1,TEN,CIN,9.800000,6.400000
13227,Division,Kansas City Chiefs,42.0,36.0,Buffalo Bills,35.0,6.0,54.0,True,True,2022,1,KC,BUF,-1.000000,3.538462
13228,Division,Tampa Bay Buccaneers,27.0,30.0,Los Angeles Rams,50.0,11.0,69.0,False,True,2022,1,TB,LAR,14.500000,-1.125000
13229,Conference,Kansas City Chiefs,24.0,27.0,Cincinnati Bengals,41.0,4.0,41.0,False,True,2022,1,KC,CIN,-7.000000,5.538462


In [59]:
humid_df = weather_df.groupby(['team_home', 'schedule_week'])['weather_humidity'].mean()

In [60]:
df2 = pd.read_csv('nfl_teams.csv')

In [61]:
df2.head()

,team_name,team_name_short,team_id,team_id_pfr,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,Arizona Cardinals,Cardinals,ARI,CRD,NFC,NFC West,NFC,NFC West
1,Phoenix Cardinals,Cardinals,ARI,CRD,NFC,NaN,NFC,NFC East
2,St. Louis Cardinals,Cardinals,ARI,ARI,NFC,NaN,NFC,NFC East
3,Atlanta Falcons,Falcons,ATL,ATL,NFC,NFC South,NFC,NFC West
4,Baltimore Ravens,Ravens,BAL,RAV,AFC,AFC North,AFC,AFC Central


In [62]:
df3 = pd.read_csv('nfl_stadiums.csv', encoding='latin_1')

In [63]:
df3.head()

,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_code,stadium_weather_type,stadium_capacity,stadium_surface,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION
0,Alamo Dome,"San Antonio, TX",NaN,NaN,indoor,"100 Montana St, San Antonio, TX 78203",78203,dome,72000,FieldTurf,NaN,NaN,NaN,NaN,NaN
1,Allegiant Stadium,"Paradise, NV",2020.0,NaN,indoor,NaN,NaN,dome,65000,Grass,NaN,NaN,NaN,NaN,NaN
2,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alumni Stadium,"Chestnut Hill, MA",NaN,NaN,outdoor,"Perimeter Rd, Chestnut Hill, MA 02467",2467,cold,NaN,Grass,NaN,NaN,NaN,NaN,NaN
4,Anaheim Stadium,"Anaheim, CA",1980.0,1994.0,outdoor,"2000 E Gene Autry Way, Anaheim, CA 92806",92806,warm,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df3['ELEVATION'].unique()

array([   nan,  264.9,  163.4,  221.9,  178. ,  189. ,    2.4,   93. ,
        193.9,  238. , 1611.2,    6.1,   15.2,   24.4,    2.1,    8.8,
        366.7,    5.8,  209.4,    3. ,   29.6,  182.9,  227.1,   47.5,
        265.8,   13.4,    1.8,  145.4,  177.7,  375.2])

In [65]:
len(df3['ELEVATION'].value_counts())

29

In [66]:
len(df3['stadium_name'].value_counts())

106

In [67]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
!pip install lxml

In [120]:
resp = requests.get('https://www.teamrankings.com/nfl/ranking/overall-power-ranking-by-team?date=2022-02-14')

In [121]:
resp.status_code

200

In [122]:
soup = BeautifulSoup(resp.content)

In [123]:
table = soup.find('table')

In [124]:
len(table)

5

In [125]:
table

<table class="tr-table datatable scrollable">
<thead>
<tr>
<th class="text-center sort-asc-first sort-first">Rank</th>
<th class="text-left sort-asc-first">Team</th>
<th class="text-right">Rating</th>
<th class="text-right nowrap">v 1-5</th>
<th class="text-right nowrap">v 6-10</th>
<th class="text-right nowrap">v 11-16</th>
<th class="text-right sort-asc-first">Hi</th>
<th class="text-right sort-asc-first">Low</th>
<th class="text-right sort-asc-first">Last</th>
</tr>
</thead>
<tbody>
<tr class="div_117 team_14">
<td class="rank tr_arrow_none text-center" data-sort="001">1</td>
<td class="nowrap" data-sort="LA Rams"><a href="https://www.teamrankings.com/nfl/team/los-angeles-rams">LA Rams</a> <small>(16-5)</small></td>
<td class="text-right" data-sort="538.6966">38.7</td>
<td class="text-right" data-sort="502">4-2</td>
<td class="text-right" data-sort="498">0-2</td>
<td class="text-right" data-sort="504">4-0</td>
<td class="text-right" data-sort="01">1</td>
<td class="text-right" data-

In [126]:
table_body = table.find('tbody')

In [127]:
rows = table_body.find_all('tr')

In [128]:
len(rows)

32

In [129]:
[td.text for td in rows[0].find_all('td')]

['1', 'LA Rams (16-5)', '38.7', '4-2', '0-2', '4-0', '1', '19', '1']

In [130]:
row_list = []
for row in table.find_all('tr'):
#     row_list.append(row.get_text().split('\n\n'))
    row_list.append([td.text for td in row.find_all('td')])
row_list[1:]

[['1', 'LA Rams (16-5)', '38.7', '4-2', '0-2', '4-0', '1', '19', '1'],
 ['2', 'San Francisco (12-8)', '35.7', '3-1', '2-2', '1-3', '2', '26', '2'],
 ['3', 'Kansas City (14-6)', '33.9', '0-2', '3-2', '4-0', '1', '19', '4'],
 ['4', 'Cincinnati (13-8)', '32.8', '2-2', '1-1', '5-0', '3', '28', '3'],
 ['5', 'Tampa Bay (14-5)', '30.8', '0-2', '3-0', '1-2', '1', '15', '5'],
 ['6', 'Green Bay (13-5)', '30.4', '3-2', '0-0', '3-2', '1', '16', '7'],
 ['7', 'Buffalo (12-7)', '30.1', '1-2', '2-1', '1-2', '2', '16', '6'],
 ['8', 'Tennessee (12-6)', '29.8', '3-1', '2-0', '4-1', '1', '26', '8'],
 ['9', 'Dallas (12-6)', '28.7', '0-3', '0-0', '2-1', '2', '26', '9'],
 ['10', 'Miami (9-8)', '27.4', '0-1', '0-3', '1-2', '6', '31', '10'],
 ['11', 'Seattle (7-10)', '27.0', '2-2', '0-2', '1-3', '4', '29', '13'],
 ['12', 'Pittsburgh (9-8-1)', '26.8', '0-4', '2-1', '1-2', '6', '26', '12'],
 ['13', 'Las Vegas (10-8)', '26.6', '0-4', '2-0', '2-0', '2', '25', '11'],
 ['14', 'Minnesota (8-9)', '26.4', '0-3', '1-2',

In [136]:
columns = table.find_all('tr')[0].get_text().split('\n')[1:10]
columns

['Rank', 'Team', 'Rating', 'v 1-5', 'v 6-10', 'v 11-16', 'Hi', 'Low', 'Last']

In [138]:
pi_2021 = pd.concat([pd.DataFrame([i], columns = columns) for i in row_list[1:]], ignore_index=True)
pi_2021

,Rank,Team,Rating,v 1-5,v 6-10,v 11-16,Hi,Low,Last
0,1,LA Rams (16-5),38.7,4-2,0-2,4-0,1,19,1
1,2,San Francisco (12-8),35.7,3-1,2-2,1-3,2,26,2
2,3,Kansas City (14-6),33.9,0-2,3-2,4-0,1,19,4
3,4,Cincinnati (13-8),32.8,2-2,1-1,5-0,3,28,3
4,5,Tampa Bay (14-5),30.8,0-2,3-0,1-2,1,15,5
5,6,Green Bay (13-5),30.4,3-2,0-0,3-2,1,16,7
6,7,Buffalo (12-7),30.1,1-2,2-1,1-2,2,16,6
7,8,Tennessee (12-6),29.8,3-1,2-0,4-1,1,26,8
8,9,Dallas (12-6),28.7,0-3,0-0,2-1,2,26,9
9,10,Miami (9-8),27.4,0-1,0-3,1-2,6,31,10


In [148]:
pi_2021['Year'] = 2021

In [149]:
pi_2021

,Rank,Team,Rating,v 1-5,v 6-10,v 11-16,Hi,Low,Last,Year
0,1,Tampa Bay (15-5),44.0,4-3,1-1,1-0,1,23,2,2021
1,2,Kansas City (16-3),38.1,4-1,3-1,3-1,1,5,1,2021
2,3,Buffalo (15-4),37.6,0-2,6-0,5-1,1,17,3,2021
3,4,Green Bay (14-4),35.9,1-2,1-0,1-1,2,14,4,2021
4,5,New Orleans (13-5),34.6,2-3,1-0,0-1,1,15,5,2021
5,6,Miami (10-6),30.4,0-3,2-1,2-1,4,28,6,2021
6,7,Baltimore (12-6),29.8,0-2,0-0,4-4,1,14,8,2021
7,8,LA Chargers (7-9),29.7,1-4,0-1,1-2,7,30,7,2021
8,9,Seattle (12-5),28.9,0-1,2-2,1-0,2,16,9,2021
9,10,LA Rams (11-7),28.9,1-2,2-2,1-0,3,21,10,2021


In [139]:
resp = requests.get('https://www.teamrankings.com/nfl/ranking/overall-power-ranking-by-team?date=2021-02-08')

In [140]:
soup = BeautifulSoup(resp.content)

In [141]:
table = soup.find('table')

In [142]:
table_body = table.find('tbody')

In [143]:
rows = table_body.find_all('tr')

In [144]:
row_list = []
for row in table.find_all('tr'):
    row_list.append([td.text for td in row.find_all('td')])
row_list[1:]

[['1', 'Tampa Bay (15-5)', '44.0', '4-3', '1-1', '1-0', '1', '23', '2'],
 ['2', 'Kansas City (16-3)', '38.1', '4-1', '3-1', '3-1', '1', '5', '1'],
 ['3', 'Buffalo (15-4)', '37.6', '0-2', '6-0', '5-1', '1', '17', '3'],
 ['4', 'Green Bay (14-4)', '35.9', '1-2', '1-0', '1-1', '2', '14', '4'],
 ['5', 'New Orleans (13-5)', '34.6', '2-3', '1-0', '0-1', '1', '15', '5'],
 ['6', 'Miami (10-6)', '30.4', '0-3', '2-1', '2-1', '4', '28', '6'],
 ['7', 'Baltimore (12-6)', '29.8', '0-2', '0-0', '4-4', '1', '14', '8'],
 ['8', 'LA Chargers (7-9)', '29.7', '1-4', '0-1', '1-2', '7', '30', '7'],
 ['9', 'Seattle (12-5)', '28.9', '0-1', '2-2', '1-0', '2', '16', '9'],
 ['10', 'LA Rams (11-7)', '28.9', '1-2', '2-2', '1-0', '3', '21', '10'],
 ['11', 'Cleveland (12-6)', '27.0', '0-1', '0-2', '4-2', '4', '27', '11'],
 ['12', 'Indianapolis (11-6)', '26.2', '1-1', '0-1', '2-3', '3', '29', '12'],
 ['13', 'Las Vegas (8-8)', '25.4', '2-3', '1-2', '1-2', '4', '23', '13'],
 ['14', 'New England (7-9)', '25.0', '0-3', '3-

In [147]:
pi_2020 = pd.concat([pd.DataFrame([i], columns = columns) for i in row_list[1:]], ignore_index=True)
pi_2020

,Rank,Team,Rating,v 1-5,v 6-10,v 11-16,Hi,Low,Last
0,1,Tampa Bay (15-5),44.0,4-3,1-1,1-0,1,23,2
1,2,Kansas City (16-3),38.1,4-1,3-1,3-1,1,5,1
2,3,Buffalo (15-4),37.6,0-2,6-0,5-1,1,17,3
3,4,Green Bay (14-4),35.9,1-2,1-0,1-1,2,14,4
4,5,New Orleans (13-5),34.6,2-3,1-0,0-1,1,15,5
5,6,Miami (10-6),30.4,0-3,2-1,2-1,4,28,6
6,7,Baltimore (12-6),29.8,0-2,0-0,4-4,1,14,8
7,8,LA Chargers (7-9),29.7,1-4,0-1,1-2,7,30,7
8,9,Seattle (12-5),28.9,0-1,2-2,1-0,2,16,9
9,10,LA Rams (11-7),28.9,1-2,2-2,1-0,3,21,10


In [155]:
list_of_url_end = ['2022-02-14','2021-02-08', '2020-02-03', '2019-02-04', '2018-02-05', '2017-02-06', 
                   '2016-02-08', '2015-02-02', '2014-01-26', '2013-02-04', '2012-02-05', '2011-02-07', 
                   '2010-02-13', '2009-02-02', '2008-02-04', '2007-02-05', '2006-02-06', '2005-02-07', '2004-02-02' ]

In [156]:
list_of_url_end

['2022-02-14',
 '2021-02-08',
 '2020-02-03',
 '2019-02-04',
 '2018-02-05',
 '2017-02-06',
 '2016-02-08',
 '2015-02-02',
 '2014-01-26',
 '2013-02-04',
 '2012-02-05',
 '2011-02-07',
 '2010-02-13',
 '2009-02-02',
 '2008-02-04',
 '2007-02-05',
 '2006-02-06',
 '2005-02-07',
 '2004-02-02']

In [158]:
for date in list_of_url_end:
    url = f"https://www.teamrankings.com/nfl/ranking/overall-power-ranking-by-team?date={date}"
    print(date[:4])

2022
2021
2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004


In [160]:
import time

In [161]:
list_of_pis = []

for date in list_of_url_end:
    url = f"https://www.teamrankings.com/nfl/ranking/overall-power-ranking-by-team?date={date}"
    resp = requests.get(url).text
    soup = BeautifulSoup(resp)
    table = soup.find('table')
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    row_list = []
    for row in table.find_all('tr'):
        row_list.append([td.text for td in row.find_all('td')])
    row_list[1:]
    df = pd.concat([pd.DataFrame([i], columns = columns) for i in row_list[1:]], ignore_index=True)
    df['Year'] = date[:4]
    list_of_pis.append(df)
    time.sleep(.1)

In [162]:
list_of_pis

[   Rank                  Team Rating v 1-5 v 6-10 v 11-16  Hi Low Last  Year
 0     1        LA Rams (16-5)   38.7   4-2    0-2     4-0   1  19    1  2022
 1     2  San Francisco (12-8)   35.7   3-1    2-2     1-3   2  26    2  2022
 2     3    Kansas City (14-6)   33.9   0-2    3-2     4-0   1  19    4  2022
 3     4     Cincinnati (13-8)   32.8   2-2    1-1     5-0   3  28    3  2022
 4     5      Tampa Bay (14-5)   30.8   0-2    3-0     1-2   1  15    5  2022
 5     6      Green Bay (13-5)   30.4   3-2    0-0     3-2   1  16    7  2022
 6     7        Buffalo (12-7)   30.1   1-2    2-1     1-2   2  16    6  2022
 7     8      Tennessee (12-6)   29.8   3-1    2-0     4-1   1  26    8  2022
 8     9         Dallas (12-6)   28.7   0-3    0-0     2-1   2  26    9  2022
 9    10           Miami (9-8)   27.4   0-1    0-3     1-2   6  31   10  2022
 10   11        Seattle (7-10)   27.0   2-2    0-2     1-3   4  29   13  2022
 11   12    Pittsburgh (9-8-1)   26.8   0-4    2-1     1-2   6  

In [164]:
full_pi = pd.concat([df for df in list_of_pis], ignore_index=True)
full_pi

,Rank,Team,Rating,v 1-5,v 6-10,v 11-16,Hi,Low,Last,Year
0,1,LA Rams (16-5),38.7,4-2,0-2,4-0,1,19,1,2022
1,2,San Francisco (12-8),35.7,3-1,2-2,1-3,2,26,2,2022
2,3,Kansas City (14-6),33.9,0-2,3-2,4-0,1,19,4,2022
3,4,Cincinnati (13-8),32.8,2-2,1-1,5-0,3,28,3,2022
4,5,Tampa Bay (14-5),30.8,0-2,3-0,1-2,1,15,5,2022
...,...,...,...,...,...,...,...,...,...,...
603,28,Washington (5-11),16.9,1-3,0-2,2-2,8,31,28,2004
604,29,LA Chargers (4-12),16.1,0-0,0-4,0-4,25,32,29,2004
605,30,Arizona (4-12),15.1,0-1,1-2,0-4,22,32,30,2004
606,31,Las Vegas (4-12),14.3,0-1,0-3,1-2,2,32,31,2004


In [174]:
pi_csv = full_pi.to_csv('pi.csv')

In [175]:
pi_csv[:100]

TypeError: 'NoneType' object is not subscriptable

In [176]:
!ls

'NFL Gambling Modeling.ipynb'		 nfl_teams.csv	 spreadspoke_scores.csv
'NFL Gambling Pre-Modeling Work.ipynb'	 pi.csv		 Untitled.ipynb
 nfl_stadiums.csv			 spreadspoke.R
